In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv('C:\Mentorship program\project\cleaned data\gurgaon_properties_post_feature_selection_v2.csv')


In [4]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3.0,2.0,2,New Property,850.0,0.0,0.0,0.0,Low,Low Floor
1,flat,sector 89,0.95,2.0,2.0,2,New Property,1226.0,1.0,0.0,0.0,Low,Mid Floor
2,flat,sohna road,0.32,2.0,2.0,1,New Property,1000.0,0.0,0.0,0.0,Low,High Floor
3,flat,sector 92,1.60,3.0,4.0,3+,Relatively New,1615.0,1.0,0.0,1.0,High,Mid Floor
4,flat,sector 102,0.48,2.0,2.0,1,Relatively New,582.0,0.0,1.0,0.0,High,Mid Floor


In [5]:
df['furnishing_type'].value_counts()

0.0    2349
1.0    1018
2.0     187
Name: furnishing_type, dtype: int64

In [6]:
df['furnishing_type'] = df['furnishing_type'].replace({0.0: 'unfurnished',1.0:'semifurnished',2.0: 'furnished'})

In [5]:
df.sample(5)

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
1857,house,sector 50,3.99,4.0,5.0,3+,New Property,4500.0,1.0,0.0,semifurnished,Medium,Mid Floor
15,flat,sector 92,1.55,3.0,4.0,3,Relatively New,1805.0,1.0,0.0,semifurnished,Medium,High Floor
1318,house,sector 57,6.21,6.0,6.0,3+,Old Property,2700.0,1.0,1.0,unfurnished,Medium,Mid Floor
3420,flat,sector 103,1.90,3.0,4.0,3+,Relatively New,3716.0,1.0,0.0,semifurnished,Low,Mid Floor
3244,flat,sector 108,0.85,3.0,3.0,2,Relatively New,1500.0,1.0,0.0,unfurnished,Medium,Low Floor


In [7]:
x = df.drop(columns=['price'])
y = df['price']

In [8]:
y_transformed = np.log1p(y)

## Ordinal Encoding

In [9]:
columns_to_encode = ['property_type','sector','balcony', 'agePossession','furnishing_type','luxury_category','floor_category']

In [9]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [10]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [11]:
kfold = KFold(n_splits=10,shuffle=True,random_state=42)
score = cross_val_score(pipeline, x, y_transformed, cv=kfold, scoring='r2')

In [12]:
score.mean(), score.std()

(0.7363096633436828, 0.03238005754429937)

In [13]:
x_train,x_test,y_train,y_test = train_test_split(x, y_transformed,test_size =0.2,random_state=42)

In [14]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', LinearRegression())])

In [15]:
y_pred = pipeline.predict(x_test)

In [16]:
y_pred = np.expm1(y_pred)

In [17]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.9463822160089357

In [18]:
def scorer(model_name, model):
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor',preprocessor),
        ('regressor', model)
        
    ])
    
    kfold = KFold(n_splits=10,shuffle=True,random_state=42)
    scores = cross_val_score(pipeline, x, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    x_train, x_test, y_train, y_test = train_test_split(x,y_transformed,test_size = 0.2,random_state=42)
    
    pipeline.fit(x_train,y_train)
    
    y_pred = pipeline.predict(x_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output

In [19]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
model_output =[]
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name,model))

In [22]:
model_output

[['linear_reg', 0.7363096633436828, 0.9463822160089357],
 ['svr', 0.7642012011196353, 0.8472636473483951],
 ['ridge', 0.7363125343993552, 0.946338774185337],
 ['LASSO', 0.05943378064493573, 1.528905986892753],
 ['decision tree', 0.776897423205174, 0.7269986683595668],
 ['random forest', 0.8813030745358625, 0.5329489997821205],
 ['extra trees', 0.867432840997368, 0.5414834807641574],
 ['gradient boosting', 0.8725388766332755, 0.5753606055758095],
 ['adaboost', 0.7558269539926985, 0.8759572400169354],
 ['mlp', 0.8088078407437298, 0.7272305724340681],
 ['xgboost', 0.8894876835260124, 0.5040475141482346]]

In [23]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mse'])

In [24]:
model_df.sort_values(['mse'])

,name,r2,mse
10,xgboost,0.889488,0.504048
5,random forest,0.881303,0.532949
6,extra trees,0.867433,0.541483
7,gradient boosting,0.872539,0.575361
4,decision tree,0.776897,0.726999
9,mlp,0.808808,0.727231
1,svr,0.764201,0.847264
8,adaboost,0.755827,0.875957
2,ridge,0.736313,0.946339
0,linear_reg,0.736310,0.946382


## OneHotEncoding

In [25]:
# creating a column transformer for preprocessing
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
    ('cat1',  OrdinalEncoder(), columns_to_encode ),
    ('cat2', OneHotEncoder(drop='first'),['sector','agePossession','furnishing_type'])
], 
    remainder = 'passthrough'
)

In [26]:
pipeline = Pipeline([
    ('preprocessor',preprocessor),
    ('regressor',LinearRegression())
])

In [27]:
kfold = KFold(n_splits=10,shuffle=True,random_state=42)
scores = cross_val_score(pipeline,x ,y_transformed, cv=kfold, scoring='r2')

In [28]:
scores.mean(),scores.std()

(0.8546151830349675, 0.01599250037818318)

In [29]:
X_train,X_test,y_train,y_test = train_test_split(x, y_transformed,test_size=0.2,random_state = 42)

In [30]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat1', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat2',
                                                  OneH

In [31]:
y_pred = pipeline.predict(X_test)

In [32]:
y_pred = np.expm1(y_pred)

In [33]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.6496724738872373

In [34]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, x, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(x,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output

In [35]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [37]:
model_df = pd.DataFrame(model_output,columns=['name','r2','mae'])

In [38]:
model_df.sort_values(['mae'])

,name,r2,mae
6,extra trees,0.893775,0.468509
10,xgboost,0.895850,0.493456
5,random forest,0.890945,0.497337
9,mlp,0.872485,0.520661
7,gradient boosting,0.876599,0.570627
0,linear_reg,0.854615,0.649672
2,ridge,0.849676,0.650245
4,decision tree,0.807823,0.690594
1,svr,0.769741,0.834124
8,adaboost,0.750156,0.845143


In [39]:
from sklearn.model_selection import GridSearchCV


In [40]:
param_grid = {
    'regressor__n_estimators': [50, 100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__max_samples':[0.1, 0.25, 0.5, 1.0],
    'regressor__max_features': ['auto', 'sqrt']
}

In [44]:
import category_encoders as ce
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first'),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)

In [43]:
!pip install category_encoders

In [45]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])



<IPython.core.display.Javascript object>

In [46]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [47]:
search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1, verbose=4)

In [48]:
search.fit(x, y_transformed)

Fitting 10 folds for each of 128 candidates, totalling 1280 fits


GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['bedRoom',
                                                                          'bathroom',
                                                                          'built_up_area',
                                                                          'servant '
                                                                          'room',
                                                                          'store '
                                                                          'room']),
                           

In [49]:
final_pipe = search.best_estimator_


In [50]:
search.best_params_

{'regressor__max_depth': None,
 'regressor__max_features': 'sqrt',
 'regressor__max_samples': 1.0,
 'regressor__n_estimators': 300}

In [51]:
search.best_score_

0.9029119975652738

In [52]:
final_pipe.fit(x,y_transformed)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1',
                                                  OneHo

## Exporting the model

In [10]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first'),['sector','agePossession'])
    ], 
    remainder='passthrough'
)

In [11]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=500))
])

<IPython.core.display.Javascript object>

In [12]:
pipeline.fit(x,y_transformed)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector',
                                                   'agePossession'])])),
                ('regressor', RandomForestRegressor(n_estimators=500))])

In [13]:
import pickle

with open('pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [14]:
with open('df.pkl', 'wb') as file:
    pickle.dump(x, file)

In [30]:
import sys
print(sys.version)


3.9.18 (main, Sep 11 2023, 14:09:26) [MSC v.1916 64 bit (AMD64)]
